In [ ]:
import re
import math
import numpy as np
from functools import reduce, cmp_to_key, partial
from itertools import accumulate

def read_input(filename="input.txt"):
    with open(filename) as f:
        a = f.readlines()
    b = (        
        list(map(int, re.findall(r"[-\d]+", l)))
        for j, l in enumerate(a)
    )
    return np.array([p for p in b], dtype=int)

def md(a):
  return np.abs(a[:, 0]-a[:, 2]) + np.abs(a[:,1]-a[:,3])

def segment(sensor, y):
    i, j, d = sensor
    dd = abs(j - y) # diff between sensor y and the inspected y
    if dd > d: # sensor is outside of range
        return None
    r = d - dd
    return [i - r, i + r]

def segmentY(sensor, x):
    i, j, d = sensor
    dd = abs(i - x) # diff between sensor y and the inspected y
    if dd > d: # sensor is outside of range
        return None
    r = d - dd
    return [j - r, j + r]


def union(a, b):
  if b[1] < a[0] or b[0] > a[1]:
    return None
  return [min(a[0], b[0]), max(a[1], b[1])]

def compare(a, b):
  if a[0] == b[0]:
    return -1 if a[1] - a[0] >= b[1] - b[0]  else 1
  return -1 if a[0] < b[0] else 1

def combine(s, x):
  u = union(s[-1], x)
  if u is None:
    return s + [x]
  s[-1] = u
  return s

A = read_input("input.txt")
H = A[:,2:]
C = np.hstack([A[:,:2], md(A).reshape((-1, 1))])

def computeX(y):
  beacons = np.unique(H[H[:,1] == y, :], axis=0)

  sg = [s for s in (segment(r, y) for r in C) if s is not None]

  ssg = list(sorted(sg, key=cmp_to_key(compare)))
  print(ssg)
  uniq = reduce(combine, ssg[1:], [ssg[0]])
  print(uniq)
  return reduce(lambda s, x: s + (x[1] - x[0] + 1), uniq, 0) - beacons.shape[0]

def computeY(x):
  beacons = np.unique(H[H[:,0] == x, :], axis=0)

  sg = [s for s in (segmentY(r, x) for r in C) if s is not None]

  ssg = list(sorted(sg, key=cmp_to_key(compare)))
  # print(ssg)
  uniq = reduce(combine, ssg[1:], [ssg[0]])
  print(uniq)
  return reduce(lambda s, x: s + (x[1] - x[0] + 1), uniq, 0) - beacons.shape[0]

print(computeX(11))
print(computeY(14))



In [ ]:
# PART II
#x_max, y_max = 20, 20
x_max, y_max = 4000000, 4000000

N = x_max + 1


def rrr(s, x):
    n = max((min(x[1], x_max) - max(x[0], 0) + 1), 0)
    return s + n 

mask = (A[:, 3] <= y_max) & (A[:, 3] >= 0) & (A[:, 2] <= x_max) & (A[:, 2] >= 0)
beacons = np.unique(A[mask, 2:], axis=0)
bs = {}
for y in np.unique(beacons[:, 1]):
    bs[y] = np.sum(beacons[:, 1] == y)
print(bs)
s_zero = 0
found = None
for y in range(y_max // 2, y_max + 1):
    sg = (s for s in (segment(r, y) for r in C) if s is not None)
    ssg = list(sorted(sg, key=cmp_to_key(compare)))
    # print(ssg)
    if y % 100000 == 0:
        print(y / y_max)
    uniq = reduce(combine, ssg[1:], [ssg[0]])
    not_available = reduce(rrr, uniq, 0) + (bs[y] if y in bs else 0)
    available = N - not_available
    assert available <= 1
    if available == 1:
      found = (y, uniq)
    if available == 0:
        s_zero += 1
    
print(s_zero)
print(y_max - s_zero)
        


In [ ]:
found

In [ ]:
y_max*(found[1][0][1]+1) + found[0]